# Data creation

In [ ]:
import json
import pandas as pd
from pathlib import Path

# ---------- CONFIG ----------
FPS = 25

from google.colab import drive
drive.mount('/content/drive')
# file_path = '/content/drive/My Drive/datasets/my_data.csv'
# df = pd.read_csv(file_path)
tracks_dir = Path("/content/drive/My Drive/tno/data/")   # folder with 01_tracks.xlsx ... 60_tracks.xlsx
output_dir = Path("/content/drive/My Drive/tno/processed/merged_tracks_multi_hot")
json_path = Path("/content/drive/My Drive/tno/data/all_scenarios.json")

output_dir.mkdir(exist_ok=True)
# ----------------------------

## Labelling highD CSV using StreetWise created frame-level labels (json file) 

In [ ]:
# Load scenarios JSON
with open(json_path, "r") as f:
    scenarios = json.load(f)

# Collect all unique scenario categories across datasets
all_categories = set()
for _, categories in scenarios.items():
    all_categories.update(categories.keys())
all_categories = sorted(all_categories)  # fixed column order

def expand_scenarios_for_dataset(categories, fps=FPS):
    """Expand one dataset's scenarios to frame-level DataFrame (multi-hot)."""
    records = []
    for category, items in categories.items():
        for item in items:
            frame_start = int(item["tstart"] * fps)
            frame_end = int(item["tend"] * fps)
            ego_id = item.get("ego_id")
            for frame in range(frame_start, frame_end + 1):
                records.append({
                    "frame": frame,
                    "id": ego_id,
                    "scenario": category
                })
    df = pd.DataFrame(records)
    if df.empty:
        return df

    # Pivot to multi-hot format
    df["value"] = 1
    df = df.pivot_table(index=["frame", "id"],
                        columns="scenario",
                        values="value",
                        fill_value=0)
    df = df.reset_index()
    return df

# Store for global concatenation
all_datasets = []

# Loop through all datasets in JSON
for dataset_id_str, categories in scenarios.items():
  if dataset_id_str in ["56"]:
    dataset_id = int(dataset_id_str)
    track_file = tracks_dir / f"{dataset_id:02d}_tracks.csv"
    if not track_file.exists():
        print(f"⚠️ Missing {track_file}, skipping")
        continue

    # Load trajectory
    tracks_df = pd.read_csv(track_file)

    # Expand JSON scenarios into multi-hot labels
    scenario_df = expand_scenarios_for_dataset(categories, fps=FPS)

    # Ensure all scenario columns exist
    for cat in all_categories:
        if cat not in scenario_df.columns:
            scenario_df[cat] = 0

    # Merge on frame + id
    merged = tracks_df.merge(scenario_df, on=["frame", "id"], how="left")

    # Fill NaN for frames with no scenario
    merged[all_categories] = merged[all_categories].fillna(0).astype(int)

    # Save per dataset
    out_path = output_dir / f"{dataset_id:02d}_tracks_with_multi_hot.csv"
    merged.to_csv(out_path, index=False)
    print(f"✅ Saved per-dataset merged file: {out_path}")

    # Add dataset_id for concatenated version
    merged["dataset_id"] = dataset_id
    all_datasets.append(merged)

## Concatenating CSV files

In [ ]:
import pandas as pd

# Pick the first 5 files
files_to_concat = [
    (i, output_dir / f"{i:02d}_tracks_with_multi_hot.csv")
    for i in range(53, 57)  # 01–05
]

dfs = []
for doc_id, f in files_to_concat:
    if f.exists():
        df = pd.read_csv(f)
        # Add document_id column first
        cols = df.columns.tolist()
        if "id" in cols:
            idx = cols.index("id")
            cols.insert(idx, "document_id")
            df["document_id"] = doc_id
            df = df[cols]  # reorder columns
        else:
            # If no 'id' column, just add at the start
            df.insert(0, "document_id", doc_id)
        dfs.append(df)
    else:
        print(f"⚠️ Missing {f}, skipping")

# Concatenate
if dfs:
    combined_5 = pd.concat(dfs, ignore_index=True)
    out_path_5 = output_dir / "53-57_tracks_with_multi_hot.csv"
    combined_5.to_csv(out_path_5, index=False)
    print(f"✅ Saved concatenated file for first 5 datasets with document_id before id: {out_path_5}")
else:
    print("⚠️ No files found to concatenate.")


## Pre-processing/cleaning CSV and converting to Parquet

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import StandardScaler
import os
import joblib
import glob # Added glob for file listing in the next script logic

fine_tuning=True
testing_f=True

tracks_dir = Path("/kaggle/input/data-tno/data")
fine_file = Path("/kaggle/input/01-03-tracks-with-multi-hot/01-03_tracks_with_multi_hot.csv")
testing_file = Path("/kaggle/input/57-61-tracks-with-multi-hot/57-61_tracks_with_multi_hot.csv")


output_dir = Path("/kaggle/working/processed_parquet")
output_dir.mkdir(exist_ok=True)

SCALER_MAIN_PATH = "/kaggle/working/scaler_main.joblib"
SCALER_PREC_PATH = "/kaggle/working/scaler_prec.joblib"

numerical_features = [
    'x','y','xVelocity','yVelocity','xAcceleration','yAcceleration',
    'frontSightDistance','backSightDistance','dhw','thw','ttc'
]

vehicle_cols = [
    'precedingId','followingId','leftPrecedingId','rightPrecedingId',
    'leftFollowingId','rightFollowingId','leftAlongsideId','rightAlongsideId'
]

scaler_main = StandardScaler()
scaler_prec = StandardScaler()

# ---------------------------
# PASS 1 — fit scalers across all 52 raw CSV files
# ---------------------------
for dataset_id in range(3, 55):
    fp = tracks_dir / f"{dataset_id:02d}_tracks.csv"
    if not fp.exists(): continue
    
    df = pd.read_csv(fp, usecols=[
        *numerical_features, 'precedingId','precedingXVelocity'
    ])

    # fix negative velocities
    mask = df["xVelocity"] < 0
    if mask.any():
        df.loc[mask, ["xVelocity","yVelocity","xAcceleration","yAcceleration","precedingXVelocity"]] *= -1

    # update main numerical scaler
    scaler_main.partial_fit(df[numerical_features])

    # preceding velocity partial fit (convert NaN to 0)
    prec = df['precedingXVelocity'].where(df['precedingId']>0, np.nan)
    scaler_prec.partial_fit(np.nan_to_num(prec.values.reshape(-1,1), nan=0))

print("✔ Scalors fitted over all 52 train files")

# ---------------------------
# NEW: Save the fitted scalers for export
# ---------------------------
try:
    joblib.dump(scaler_main, SCALER_MAIN_PATH)
    joblib.dump(scaler_prec, SCALER_PREC_PATH)
    print(f"✔ Scaler_main exported to: {SCALER_MAIN_PATH}")
    print(f"✔ Scaler_prec exported to: {SCALER_PREC_PATH}")
except Exception as e:
    print(f"❌ Error exporting scalers: {e}")


# ---------------------------
# PASS 2 — PROCESS AND SAVE TRAIN FILES (Features only, with 0, 1, ... 19 columns)
# ---------------------------
for dataset_id in range(3, 55):
# for dataset_id in range(3, 15):
    fp = tracks_dir / f"{dataset_id:02d}_tracks.csv"
    if not fp.exists(): continue
    
    df = pd.read_csv(fp)

    # negative velocity fix
    mask = df["xVelocity"] < 0
    if mask.any():
        df.loc[mask, ["xVelocity","yVelocity","xAcceleration","yAcceleration","precedingXVelocity"]] *= -1

    # lane remapping
    unique_lanes = set(df["laneId"].unique())
    if unique_lanes == {2,3,5,6}:
        df['laneId'] = df['laneId'].replace({2:6, 3:5})
    elif unique_lanes == {2,3,4,6,7,8}:
        df['laneId'] = df['laneId'].replace({2:8,3:7,4:6})

    # Build features
    X = df[numerical_features].copy()

    # add binary indicators
    for col in vehicle_cols:
        X['has_'+col.replace("Id","")] = (df[col] > 0).astype(int)

    # scale numerical
    X_scaled_num = scaler_main.transform(X[numerical_features])

    # scale preceding velocity
    prec = df['precedingXVelocity'].where(df['precedingId']>0, np.nan)
    prec_scaled = scaler_prec.transform(np.nan_to_num(prec.values.reshape(-1,1), nan=0))

    # binary cols
    binary_cols = [c for c in X.columns if c.startswith("has_")]

    # final stacked feature matrix (20 features)
    final = np.hstack([X_scaled_num, prec_scaled, X[binary_cols].values])
    
    # --- START OF CHANGE FOR TRAIN FILES ---
    # Create DataFrame WITHOUT providing column names. Pandas uses 0, 1, 2, ...
    pd.DataFrame(final).to_parquet( # Removed columns=out_cols
        output_dir / f"{dataset_id:02d}_tracks_preprocessed.parquet",
        index=False
    )
    # --- END OF CHANGE FOR TRAIN FILES ---

    print("✔ Saved train parquet:", dataset_id)


# ---------------------------
# PROCESS AND SAVE FINE-TUNE FILE (Features + Labels)
# ---------------------------

if fine_tuning:
    df_main = pd.read_csv(fine_file)
    # target_cols = df_main.columns[26:38]
    target_cols=df_main.columns[26:38].delete([28-26, 33-26])
    
    df = df_main[~df_main[target_cols].eq(0).all(axis=1)].copy()
    
    # negative velocities
    mask = df["xVelocity"] < 0
    if mask.any():
        df.loc[mask, ["xVelocity","yVelocity","xAcceleration","yAcceleration","precedingXVelocity"]] *= -1
    
    # lane remapping
    unique_lanes = set(df["laneId"].unique())
    if unique_lanes == {2,3,5,6}:
        df['laneId'] = df['laneId'].replace({2:6,3:5})
    elif unique_lanes == {2,3,4,6,7,8}:
        df['laneId'] = df['laneId'].replace({2:8,3:7,4:6})
    
    # Features
    X = df[numerical_features].copy()
    for col in vehicle_cols:
        X['has_'+col.replace("Id","")] = (df[col] > 0).astype(int)
    
    # scale numerical
    X_scaled_num = scaler_main.transform(X[numerical_features])
    
    # scale preceding velocity
    prec = df['precedingXVelocity'].where(df['precedingId']>0, np.nan)
    prec_scaled = scaler_prec.transform(np.nan_to_num(prec.values.reshape(-1,1), nan=0))
    
    binary_cols = [c for c in X.columns if c.startswith("has_")]
    
    # labels
    # scenario_cols = df.columns[26:38]
    # Y = df[scenario_cols].values
    scenario_cols = target_cols
    Y = df[scenario_cols].values
    
    # final matrix features are stacked first
    final_matrix = np.hstack([
        df[['document_id','id','frame']].values, # 3 columns
        X_scaled_num,                            # 11 numerical + 1 prec_scaled = 12 columns
        prec_scaled,                             # Combined into X_scaled_num above (1 column)
        X[binary_cols].values,                   # 8 binary columns
        Y                                        # 12 label columns
    ])
    
    # The feature columns (0 to 19) are X_scaled_num (11) + prec_scaled (1) + X[binary_cols].values (8)
    
    # The total number of columns in final_matrix is 3 (IDs) + 20 (Features) + 12 (Labels) = 35
    
    # --- START OF CHANGE FOR FINE-TUNE FILE ---
    # 1. Define column names ONLY for the non-feature data (IDs and Labels).
    id_cols = ['document_id','id','frame']
    label_cols = list(scenario_cols)
    
    # 2. Generate the integer column names for the 20 features (0 to 19)
    feature_cols = [i for i in range(20)]
    
    # 3. Create the final column list
    final_cols = id_cols + feature_cols + label_cols
    
    # 4. Use the combined list to create the DataFrame
    df_out = pd.DataFrame(final_matrix, columns=final_cols)
    df_out.to_parquet(output_dir / "fine_tuning_preprocessed.parquet", index=False)
    # --- END OF CHANGE FOR FINE-TUNE FILE ---
    
    print("✔ Saved FINE-TUNE parquet")

if testing_f:
    df_main = pd.read_csv(testing_file)
    # target_cols = df_main.columns[26:38]
    target_cols=df_main.columns[26:38].delete([28-26, 33-26])
    
    df = df_main[~df_main[target_cols].eq(0).all(axis=1)].copy()
    
    # negative velocities
    mask = df["xVelocity"] < 0
    if mask.any():
        df.loc[mask, ["xVelocity","yVelocity","xAcceleration","yAcceleration","precedingXVelocity"]] *= -1
    
    # lane remapping
    unique_lanes = set(df["laneId"].unique())
    if unique_lanes == {2,3,5,6}:
        df['laneId'] = df['laneId'].replace({2:6,3:5})
    elif unique_lanes == {2,3,4,6,7,8}:
        df['laneId'] = df['laneId'].replace({2:8,3:7,4:6})
    
    # Features
    X = df[numerical_features].copy()
    for col in vehicle_cols:
        X['has_'+col.replace("Id","")] = (df[col] > 0).astype(int)
    
    # scale numerical
    X_scaled_num = scaler_main.transform(X[numerical_features])
    
    # scale preceding velocity
    prec = df['precedingXVelocity'].where(df['precedingId']>0, np.nan)
    prec_scaled = scaler_prec.transform(np.nan_to_num(prec.values.reshape(-1,1), nan=0))
    
    binary_cols = [c for c in X.columns if c.startswith("has_")]
    
    # labels
    # scenario_cols = df.columns[26:38]
    # Y = df[scenario_cols].values
    scenario_cols = target_cols
    Y = df[scenario_cols].values
    
    # final matrix features are stacked first
    final_matrix = np.hstack([
        df[['document_id','id','frame']].values, # 3 columns
        X_scaled_num,                            # 11 numerical + 1 prec_scaled = 12 columns
        prec_scaled,                             # Combined into X_scaled_num above (1 column)
        X[binary_cols].values,                   # 8 binary columns
        Y                                        # 12 label columns
    ])
    
    # The feature columns (0 to 19) are X_scaled_num (11) + prec_scaled (1) + X[binary_cols].values (8)
    
    # The total number of columns in final_matrix is 3 (IDs) + 20 (Features) + 12 (Labels) = 35
    
    # --- START OF CHANGE FOR FINE-TUNE FILE ---
    # 1. Define column names ONLY for the non-feature data (IDs and Labels).
    id_cols = ['document_id','id','frame']
    label_cols = list(scenario_cols)
    
    # 2. Generate the integer column names for the 20 features (0 to 19)
    feature_cols = [i for i in range(20)]
    
    # 3. Create the final column list
    final_cols = id_cols + feature_cols + label_cols
    
    # 4. Use the combined list to create the DataFrame
    df_out = pd.DataFrame(final_matrix, columns=final_cols)
    df_out.to_parquet(output_dir / "testing_preprocessed.parquet", index=False)
    # --- END OF CHANGE FOR FINE-TUNE FILE ---
    
    print("✔ Saved testing parquet")

✔ Scalors fitted over all 52 train files
✔ Scaler_main exported to: /kaggle/working/scaler_main.joblib
✔ Scaler_prec exported to: /kaggle/working/scaler_prec.joblib
✔ Saved train parquet: 3
✔ Saved train parquet: 4
✔ Saved train parquet: 5
✔ Saved train parquet: 6
✔ Saved train parquet: 7
✔ Saved train parquet: 8
✔ Saved train parquet: 9
✔ Saved train parquet: 10
✔ Saved train parquet: 11
✔ Saved train parquet: 12
✔ Saved train parquet: 13
✔ Saved train parquet: 14
✔ Saved train parquet: 15
✔ Saved train parquet: 16
✔ Saved train parquet: 17
✔ Saved train parquet: 18
✔ Saved train parquet: 19
✔ Saved train parquet: 20
✔ Saved train parquet: 21
✔ Saved train parquet: 22
✔ Saved train parquet: 23
✔ Saved train parquet: 24
✔ Saved train parquet: 25
✔ Saved train parquet: 26
✔ Saved train parquet: 27
✔ Saved train parquet: 28
✔ Saved train parquet: 29
✔ Saved train parquet: 30
✔ Saved train parquet: 31
✔ Saved train parquet: 32
✔ Saved train parquet: 33
✔ Saved train parquet: 34
✔ Saved tr

## Zip the parquet files for downloading

In [2]:
import shutil
shutil.make_archive("/kaggle/working/processed_parquet_zip", 'zip', "/kaggle/working/processed_parquet")
print("✅ Zipped processed_parquet folder")

✅ Zipped processed_parquet folder
